# Re-create OpenAI's Deep Research using OpenAI Agents SDK

### Four different agents will be made:

1. Search Agent

    - Takes a search term from *Planner agent* and then searches online for that term
    - Uses `WebSearchTool` (one of OpenAI Agents SDK's *hosted tools*)

2. Planner Agent

    - Takes a general question from a user and then forms suitable searches based on it

3. Report Agent

    - Takes data gathered from searches and then turns it into a summary report

4. Push Agent

    - Sends a notification to the user's phone with the summary

In [50]:
# Import packages and modules
import os
import requests
import asyncio

from dotenv import load_dotenv
from agents import Agent, trace, Runner, WebSearchTool, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from IPython.display import display, Markdown
from pprint import pprint

In [51]:
# Initialize constants
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [52]:
# Load the OpenAI API key
load_dotenv(override=True)

True

## Search agent

When given a search term, search the internet for it, and finally return a summary of the results.

In [53]:
# System prompt
INSTRUCTIONS = "You're a research assistant. When given a search term, search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words long. Capture the main points. Write succinctly, no need to do complete sentences or good \
grammar. The summary will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Don't include any additional commentary other than the summary itself."

# Create the search agent
# Arm it with `WebSearchTool`
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    # Make sure the agent uses the tool
    model_settings=ModelSettings(tool_choice="required"),
)

In [54]:
# User query
message = "What are the most popular and successful AI Agent frameworks in November 2025"

# Set up tracing
with trace("Search"):
    # Run the search agent
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of November 2025, several AI agent frameworks have gained prominence for their capabilities and adoption:

- **LangChain**: A versatile framework for building context-aware, reasoning applications, known for its extensive integrations and reusable components. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-trends-popularity-2025-sachin-tiwari-c5tuc?utm_source=openai))

- **LangGraph**: An extension of LangChain, focusing on graph-based orchestration for managing long-running, stateful agents with complex branching. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **AutoGen**: Developed by Microsoft, this framework emphasizes multi-agent collaboration and asynchronous task execution with human-in-the-loop oversight. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **CrewAI**: A rapidly growing framework that prioritizes multi-agent collaboration and workflow design, noted for its user-friendly interface suitable for prototyping. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **Semantic Kernel**: Also from Microsoft, this open-source development kit is tailored for building enterprise-grade generative AI applications. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework focusing on creating multi-agent workflows with comprehensive tracing and guardrails, compatible with over 100 different LLMs. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: A modular framework integrating with Google's ecosystem, supporting hierarchical agent compositions with minimal code requirements. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **$Agent^2$**: An agent-generates-agent framework for reinforcement learning automation, enabling intelligent LLM-driven generation of RL agents without human intervention. ([arxiv.org](https://arxiv.org/abs/2509.13368?utm_source=openai))

- **AutoAgent**: A fully-automated, zero-code framework for LLM agents, allowing users to create and deploy agents through natural language alone. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system for intelligent experience entities, featuring a standardized Agent-to-Agent communication layer built on the Model Context Protocol (MCP). ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

These frameworks are shaping the AI agent landscape, offering diverse tools for developing intelligent, autonomous systems. 

### Go look at the trace

https://platform.openai.com/traces

## Planner agent

When given a query, come up with a set of ideas for web searches that could be run.

In [ ]:
NUM_SEARCHES = 5

# System prompt
INSTRUCTIONS = f"You're a research assistant. When given a query, come up with a set of web searches \
to perform to best answer the query. Output {NUM_SEARCHES} terms to query for."

# Use Pydantic objects to describe the Schema of the output
# This is sometimes called 'structured outputs'

# 
class WebSearchItem(BaseModel):
    # We want the agent to tell us why a search is important,
    # this way we get better outcomes

    # This field forces the agent to operate in a reasoning mode
    # and tell you why it's doing what it's doing
    # before telling you the answer
    # This biases it to be more likely to output tokens
    # consistent with its reasoning
    reason: str
    "Your reasoning for why this search is important to the query."

    # The actual search term
    query: str
    "The search term to use for the web search."

# A list of web searches to perform (a plan)
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# Create the planner agent
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    # Passing the plan here ensures the output follows the schema
    output_type=WebSearchPlan,
)

In [ ]:
# User query
message = "What are the most popular and successful AI Agent frameworks in November 2025"

# Set up tracing
with trace("Search"):
    # Run the planner agent
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

# Now the agent presents us five queries with reasoning behind them

WebSearchPlan(searches=[WebSearchItem(reason='To find the most popular AI agent frameworks currently in use.', query='popular AI agent frameworks 2025'), WebSearchItem(reason='To discover which AI agent frameworks are considered the most successful or impactful in 2025.', query='successful AI agent frameworks November 2025'), WebSearchItem(reason='To identify any new or emerging AI agent frameworks as of late 2025.', query='new AI agent frameworks 2025'), WebSearchItem(reason='To find expert reviews or comparisons of AI agent frameworks relevant to 2025.', query='AI agent frameworks comparison 2025'), WebSearchItem(reason='To gather community feedback or user adoption statistics for AI agent frameworks in 2025.', query='AI agent frameworks user adoption statistics 2025')])
